# Is there any relationship between suicides and forest area?

Required tools and libraries in this analysis:

In [12]:
import pandas as pd
import basedosdados as bd
from matplotlib import pyplot as plt

%matplotlib inline

## Available data
### To approach this question we'll consider the following datasets:
#### 1. [Sistema de Informações sobre Mortalidade (SIM)](https://basedosdados.org/dataset/br-ms-sim?bdm_table=microdados)
CID-10 codes of interest: X60-X84 e Y87.0 

#### 2. [Censo Agropecuário](https://basedosdados.org/dataset/br-ibge-censo-agropecuario?bdm_table=municipio)


#### 3. [População brasileira](https://basedosdados.org/dataset/br-ms-populacao?bdm_table=municipio)


### References:
- [1] [Mortalidade por suicídio e notificações de lesões autoprovocadas no Brasil](https://www.gov.br/saude/pt-br/centrais-de-conteudo/publicacoes/boletins/epidemiologicos/edicoes/2021/boletim_epidemiologico_svs_33_final.pdf)

## Simplest case: municipios, 2007 
At first, lets focus on the year of 2017, as it is covered by all datasets and the most recent available Censo Agropecuário.

In [25]:
class Get_data:
    def _unpack(self, l):
        return ", ".join(map(str, l))

    def _get_data(self, filename, query):
        try:
            return pd.read_csv(f'../data/{filename}.csv')
        except FileNotFoundError:
            df = bd.read_sql(query=query,
                             billing_project_id='explorando-basedosdados')
            df.to_csv(
                f'../data/{filename}.csv', index=False)
            return df

    def sim_suic_municipio_2017(self):
        filename = 'sim_suic_municipio_2017'
        columns = ('id_municipio', 'SUM(numero_obitos) as numero_obitos')
        query = f"""
                SELECT 
                    {self._unpack(columns)}
                FROM 
                    br_ms_sim.municipio_causa
                WHERE 
                    ano = 2017
                    AND (
                        CONTAINS_SUBSTR(causa_basica, 'X6')
                        OR CONTAINS_SUBSTR(causa_basica, 'X7')
                        OR CONTAINS_SUBSTR(causa_basica, 'X8')
                        OR causa_basica = 'Y870'
                        )
                    GROUP BY id_municipio
                """
        return self._get_data(filename, query)

    def get_agro_forest_municipio_2017(self):
        filename = 'agro_forest_municipio_2017'
        columns = ('id_municipio',
                   'area_total',
                   'area_mata_natural',
                   'area_mata_plantada',
                   'area_sistema_agroflorestal',
                   'area_mata')
        query = f"""
                SELECT 
                    {self._unpack(columns)}
                FROM 
                    br_ibge_censo_agropecuario.municipio
                WHERE
                    ano = 2017
                    """
        return self._get_data(filename, query)


get_data = Get_data()
sim_suic_2017 = get_data.sim_suic_municipio_2017()
agro_forest_2017 = get_data.get_agro_forest_municipio_2017()


In [21]:
sim_suic_2017

,id_municipio,numero_obitos
0,1600303,31
1,1600402,2
2,1600535,1
3,1300029,1
4,1300144,2
...,...,...
3139,3556305,1
3140,3556354,1
3141,3556800,1
3142,3557105,7


In [22]:
agro_forest_2017

,id_municipio,area_total,area_mata_natural,area_mata_plantada,area_sistema_agroflorestal,area_mata
0,1100015,372746.0,108944.0,258.0,10481.0,109202.0
1,1100023,334256.0,88865.0,615.0,1573.0,89480.0
2,1100031,113085.0,17271.0,593.0,0.0,17864.0
3,1100049,221390.0,31620.0,180.0,0.0,31800.0
4,1100056,126686.0,27954.0,0.0,489.0,27954.0
...,...,...,...,...,...,...
5558,5222005,79967.0,12186.0,715.0,0.0,12901.0
5559,5222054,62341.0,5304.0,0.0,2626.0,5304.0
5560,5222203,84770.0,5542.0,0.0,4663.0,5542.0
5561,5222302,186118.0,34659.0,724.0,1083.0,35383.0


In [24]:
df = pd.merge(sim_suic_municipio_2017,agro_forest_municipio_2017,how='left',on='id_municipio')
df

,id_municipio,numero_obitos,area_total,area_mata_natural,area_mata_plantada,area_sistema_agroflorestal,area_mata
0,1600303,31,130310.0,46952.0,0.0,7197.0,46952.0
1,1600402,2,88157.0,66039.0,54.0,4590.0,66093.0
2,1600535,1,81200.0,52515.0,0.0,359.0,52515.0
3,2700300,12,13195.0,57.0,17.0,5.0,74.0
4,2700607,1,1612.0,134.0,0.0,0.0,134.0
...,...,...,...,...,...,...,...
3139,3556305,1,79523.0,9774.0,0.0,187.0,9774.0
3140,3556354,1,7281.0,1041.0,907.0,31.0,1948.0
3141,3556800,1,19792.0,1163.0,5.0,0.0,1168.0
3142,3557105,7,27907.0,2176.0,227.0,0.0,2403.0
